In [21]:
from tensorflow import keras
import tensorflow as tf

In [2]:
model = keras.models.load_model("EfficientNet.h5")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, None, None, 3 7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, None, None, 3 0           normalization[0][0]              
______________________________________________________________________________________________

block5f_expand_conv (Conv2D)    (None, None, None, 1 301056      block5e_add[0][0]                
__________________________________________________________________________________________________
block5f_expand_bn (BatchNormali (None, None, None, 1 5376        block5f_expand_conv[0][0]        
__________________________________________________________________________________________________
block5f_expand_activation (Acti (None, None, None, 1 0           block5f_expand_bn[0][0]          
__________________________________________________________________________________________________
block5f_dwconv (DepthwiseConv2D (None, None, None, 1 33600       block5f_expand_activation[0][0]  
__________________________________________________________________________________________________
block5f_bn (BatchNormalization) (None, None, None, 1 5376        block5f_dwconv[0][0]             
__________________________________________________________________________________________________
block5f_ac

In [3]:
for layer in model.layers:
     layer.trainable = False
        
droupout = model.layers[-2].output
prediction_layer = keras.layers.Dense(units=2,activation="softmax",name="predictions")(droupout)

In [9]:
customized_model = keras.Model(inputs=model.input,outputs=prediction_layer)
customized_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Total params: 64,102,809
Trainable params: 5,122
Non-trainable params: 64,097,687
__________________________________________________________________________________________________


In [10]:
import keras.metrics
customized_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',
            metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

In [14]:
from PIL import Image
from numpy import asarray
from os import listdir
import numpy as np
from sklearn.model_selection import train_test_split

images = []
labels = []
size = 100
for file in listdir('face_mask'):
    image  = Image.open('face_mask/'+file)
#     image = image.convert('L')
    resized_image = image.resize((size,size))
    data = asarray(resized_image)
    if data.shape == (size,size,3):
        images.append(data)
        labels.append(1)

for file in listdir('face'):
    image  = Image.open('face/'+file)
#     image = image.convert('L')
    resized_image = image.resize((size,size))
    data = asarray(resized_image)
    if data.shape == (size,size,3):
        images.append(data)
        labels.append(0)

x = np.array(images)
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
X_train = X_train.reshape((X_train.shape[0],size,size,3)).astype('float32')/255
X_test = X_test.reshape((X_test.shape[0],size,size,3)).astype('float32')/255

from keras.utils import np_utils

print(X_train.shape)

ytrain = np_utils.to_categorical(y_train)
ytest = np_utils.to_categorical(y_test)
print(ytrain.shape)

(1279, 100, 100, 3)
(1279, 2)


In [22]:
customized_model.fit(X_train, ytrain, epochs= 15, batch_size = 100)     

Epoch 1/15


TypeError: 'NoneType' object is not callable

In [17]:
model.evaluate(X_test, ytest)

ValueError: in user code:

    /home/fara/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1323 test_function  *
        return step_function(self, iterator)
    /home/fara/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1314 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/fara/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/fara/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/fara/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/fara/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/